# Snapping Points to Road Networks

This notebook provides several examples on how points can be snapped to road networks. It is possible to snap mapillary points to either open street map segments, or any user supplied network topology. If a user-supplied topology is given, then it must be in the form of a geopandas dataframe with a `LineString` geometry column.

In [4]:
# note, to load the local module, it might be necessary to add the repository directory to the sys path:
import os
import sys
sys.path.insert(0, os.path.abspath(".."))

In [5]:
from src.controller import MapillaryImage
from src.utils import DataUtils
from src.visualizer import Visualize

After loading the necessary libraries for these examples, we will set the following required environmental variables. Please replace the values within `<>` with the appropriate values.

In [1]:
%env DATABASE_URL=<DATABASE_URL>
%env SERVICE_ACCOUNT_PRIVATE_KEY=<PATH_TO_PRIVATE_KEY>

env: DATABASE_URL=<DATABASE_URL>
env: SERVICE_ACCOUNT_PRIVATE_KEY=<PATH_TO_PRIVATE_KEY>


## Creating and Saving Snapped Geometries

First, we will create snapped geometries and save these records to the database. The first step is to initialize the `MappilaryImage` class. This will open the connection to the database. Then, we can query some data to work with. In the following examples, we will query by image ID.

In [4]:
imgs = MapillaryImage()
gdf = imgs.select_by_image_id(804124637170637)

We can map this using the `Visualize` class:

In [ ]:
vis = Visualize(gdf)
vis.map()

The `DataUtils` class provides functions to snap points to road networks using either OSM or a user-supplied network topology. These functions also provide options to write these points to the database and how to handle conflicts. Thus, to snap points, we need to initialize the `DataUtils` class with the data that we want to snap:

In [ ]:
dutil = DataUtils(gdf)

### Snapping to an OSM Network

To snap to an OSM network, we only need to call the `snap_to_road_network` function and set `use_osm=True`.

In [ ]:
gdf_osm = dutil.snap_to_road_network(100, gdf, update_db=False, use_osm=True)

This will snap the point to the nearest OSM road within 100m. By default, when snapping to OSM, it will incluse all network types (roads, paths, etc), including private ones.

The returned geodataframe only includes the related image_id, a flag to record if OSM was used, and the geometry. To visualize this, we will need to merge this data with the original dataframe. We can also map both the snapped geometry and the original geometry using the `additional_geometries` parameter od the `map` function.

In [ ]:
merged = gdf[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(gdf_osm.rename({'image_id': 'id', 'geometry': 'osm_geometry'}, axis=1), left_on='id', right_on='id', how='right')

vis = Visualize(merged)
vis.map(additional_geometries=['osm_geometry'])

### Snapping to a Provided Network

To snap the mapillary points to a user-supplied network, we first need to create a geodataframe that includes these gemoetries. The following reads all shapefiles in a given directory. Then, we can snap to the road network as above and visualize and compare with the osm snapped results.

In [5]:
import geopandas as gp
import os

roads = None
shp_dir = "/Users/iosefa/downloads/roads/"
for file in os.listdir(shp_dir):
    if file.endswith(".shp"):
        tmp = gp.read_file(os.path.join(shp_dir, file)).to_crs('EPSG:4326')
        roads = gp.pd.concat([roads, tmp])

In [ ]:
gdf_snapped = dutil.snap_to_road_network(100, gdf, update_db=False, use_osm=False, network=roads)
merged = gdf[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(gdf_snapped.rename({'image_id': 'id', 'geometry': 'snapped_geometry'}, axis=1), left_on='id', right_on='id', how='right').merge(gdf_osm.rename({'image_id': 'id', 'geometry': 'osm_geometry'}, axis=1), left_on='id', right_on='id', how='right')

vis = Visualize(merged)
vis.map(additional_geometries=['snapped_geometry', 'osm_geometry'])

### test snapping one sequence images

In [ ]:
sequence_ids = imgs.get_sequence_ids()
df = imgs.select_by_sequence_id(sequence_ids[0])

In [ ]:
df_snapped = dutil.snap_to_road_network(10, df, update_db=False, use_osm=False, network=roads)
merged = df[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(df_snapped.rename({'image_id': 'id', 'geometry': 'snapped_geometry'}, axis=1), left_on='id', right_on='id', how='right')

vis = Visualize(merged)
vis.map(additional_geometries=['geometry','snapped_geometry'])

### Saving and Loading Snapped Geometries to the Database

When creating a snapped dataframe, it is also possible to save the resulting geometries to the database, which can then be queried later without the need to re-snap the geometries. Setting the `update_db` parameter to `True` will safe the resulting dataframe to the database and setting `update_conflicting=True` will tell the function to overwrite any existing snapped image_id geometries. This may be useful when changing the snapped to geometry to use a user supplied road network instead of OSM, for example.

In [ ]:
dutil.snap_to_road_network(100, gdf, update_db=True, use_osm=True, update_conflicting=True)

When we query data, we can choose to include snapped geometries in the returned table using the `include_snapped` parameter:

In [ ]:
imgs.select_by_image_id(804124637170637, include_snapped=True)

Now, should we decide to use our own road network and update the database with it, we would:

In [ ]:
dutil.snap_to_road_network(100, gdf, update_db=True, use_osm=False, network=roads, update_conflicting=True)

Querying the database, we can see that the snapped geometry was updated and we can visualize all three geometries: default, computed, and snapped:

In [ ]:
gdf = imgs.select_by_image_id(804124637170637, include_snapped=True)
vis = Visualize(gdf)
vis.map(additional_geometries=['geometry', 'computed_geometry', 'snapped_geometry'])

# RUN snapping to all data! 

In [7]:
sequence_ids = imgs.get_sequence_ids()
imgs = MapillaryImage()
for i in range(len(sequence_ids)):
    print(sequence_ids[i])
    df = imgs.select_by_sequence_id(sequence_ids[i])
    dutil = DataUtils(df)
    df_snapped = dutil.snap_to_road_network(10, df, update_db=True, use_osm=False, network=roads, update_conflicting=True)


QixEq2SLkvCkPo96qfTjlQ        
5kdx1SQbc6_3KzjJCdJDPg        


AttributeError: 'NoneType' object has no attribute 'wkt'

In [8]:
df = imgs.select_by_sequence_id('5kdx1SQbc6_3KzjJCdJDPg')
dutil = DataUtils(df)
df_snapped = dutil.snap_to_road_network(10, df, update_db=False, use_osm=False, network=roads, update_conflicting=False)


In [15]:
print(df.shape)
print(df_snapped['geometry'].values.tolist())

(500, 17)
[<POINT (140.985 42.389)>, <POINT (140.985 42.387)>, <POINT (140.986 42.388)>, <POINT (140.984 42.389)>, <POINT (140.985 42.387)>, <POINT (140.984 42.388)>, <POINT (140.986 42.389)>, <POINT (140.987 42.389)>, <POINT (140.986 42.389)>, <POINT (140.984 42.387)>, <POINT (140.985 42.387)>, <POINT (140.986 42.388)>, <POINT (140.984 42.388)>, <POINT (140.984 42.388)>, <POINT (140.984 42.389)>, <POINT (140.986 42.388)>, <POINT (140.984 42.388)>, <POINT (140.985 42.388)>, <POINT (140.986 42.389)>, <POINT (140.984 42.388)>, <POINT (140.984 42.388)>, <POINT (140.984 42.388)>, <POINT (140.986 42.388)>, <POINT (140.985 42.388)>, <POINT (140.985 42.387)>, <POINT (140.985 42.389)>, <POINT (140.984 42.388)>, <POINT (140.984 42.388)>, <POINT (140.985 42.387)>, <POINT (140.986 42.388)>, <POINT (140.984 42.388)>, <POINT (140.985 42.388)>, <POINT (140.984 42.387)>, <POINT (140.985 42.387)>, <POINT (140.985 42.387)>, <POINT (140.986 42.388)>, <POINT (140.985 42.387)>, <POINT (140.986 42.388)>, <

In [10]:
merged = df[['id', 'seq', 'camera_type', 'image_url', 'geometry']].merge(df_snapped.rename({'image_id': 'id', 'geometry': 'snapped_geometry'}, axis=1), left_on='id', right_on='id', how='right')
merged

,id,seq,camera_type,image_url,geometry,snapped_geometry,use_osm
0,518456039564350,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98457 42.38890),POINT (140.98458 42.38890),False
1,189431889702066,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98493 42.38726),POINT (140.98493 42.38726),False
2,291621625972829,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98620 42.38847),POINT (140.98617 42.38849),False
3,1638308459698633,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98437 42.38865),POINT (140.98438 42.38865),False
4,1191133854691693,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98501 42.38733),POINT (140.98501 42.38733),False
...,...,...,...,...,...,...,...
495,303043051293985,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (141.00286 42.39251),POINT (141.00287 42.39251),False
496,126666546115263,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.99662 42.39050),POINT (140.99661 42.39056),False
497,965367387594629,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (140.98549 42.39127),POINT (140.98554 42.39129),False
498,3842588242524848,5kdx1SQbc6_3KzjJCdJDPg,fisheye,https://storage.cloud.google.com/sudb_images/i...,POINT (141.00229 42.39185),POINT (141.00231 42.39184),False


In [ ]:

vis = Visualize(merged)
vis.map(additional_geometries=['snapped_geometry'])